In [9]:
import configparser
import os

In [10]:
os.environ['SPOTIPY_CLIENT_ID'] = '9428d146ee174ef1bc0df1bbfe26584a'
os.environ['SPOTIPY_CLIENT_SECRET'] = '5ee6c6887834490eaf1c9e808a548cc3'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:8888/callback'

In [11]:
# NOTE: this is a hacky workaround since this is just an application for private use. Not a best practise
#os.environ['OAUTH_CODE'] = config['SPOTIFY']['OAUTH_CODE']

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOauthError

class SpotifySearch:
    def __init__(self, creds=None):
        """
        Use OAuth creds if passed else a client with only API keys
        """
        self.spotipy = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
        if creds:
            self.spotipy = spotipy.Spotify(auth_manager=creds)

    def search(self, search_query):
        """
        Search through Spotify search API and return results
        """
        return self.spotipy.search(search_query)

    def this_user_id(self):
        """
        Get ID of current user - tied to OAuth
        """
        return self.spotipy.me()['id']    

    def get_first_track_extended(self, search_query):
        """
        Returns uri, api_url, id and url of first result for a specific search query
        """
        results = self.search(search_query)
        res = {}
        if len(results) > 0:
            first = results['tracks']['items'][0]
            res['url'] = first['external_urls']['spotify']
            res['api_url'] = first['href']
            res['uri'] = first['uri']
            res['id'] = first['id']
        return res

    def get_playlist_extended(self, playlist_name):
        """
        Return playlist details that match a specific playlist name and is owned by the user
        """
        playlists = self.spotipy.current_user_playlists()['items']
        username = self.spotipy.me()['display_name']

        playlist_store = {}
        for playlist in playlists:
            if playlist['name'] == playlist_name and playlist['owner']['display_name'] == username:
                playlist_store = playlist
                break

        if len(playlist_store) > 0:
            return {
                'id': playlist_store['id'],
                'name': playlist_store['name'],
                'uri': playlist_store['uri'],
            }

        return {}

    def get_first_uri(self, search_query):
        """
        Return first uri from results for a specific search query
        """
        try:
            return self.get_first_track_extended(search_query=search_query)['uri']
        except KeyError as e:
            print(f'[error] URLs/URIs not found for {search_query}')
            return {}

    def get_playlist_id(self, playlist_name):
        """
        Return ID for a specific playlist if exists and is owned by user
        """
        try:
            return self.get_playlist_extended(playlist_name=playlist_name)['id']
        except KeyError as e:
            print(f'[error] Playlist not found for {playlist_name}')
            return {}

    def add_to_playlist(self, playlist, track_uris):
        """
        Add track(s) to a playlist
        """
        self.spotipy.user_playlist_add_tracks(
            user=self.this_user_id(),
            playlist_id=playlist,
            tracks=track_uris
        )

In [13]:
scopes = [
    'playlist-modify-public', 'playlist-modify-private',
    'playlist-read-collaborative', 'playlist-read-private',
    'user-library-read', 'user-library-modify'
]
scope = ' '.join(scopes)

In [14]:
from spotipy.oauth2 import SpotifyOAuth
oauth_creds = SpotifyOAuth(scope=scope, username='tz6i0bkb9uc2ihu2cv5cnpqm5')

In [15]:
spotify = SpotifySearch(creds=oauth_creds)

In [16]:
spotify.get_playlist_extended('Meh')

OSError: [Errno 48] Address already in use